In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


# Importing Data

In [2]:
BBB_Train = pd.read_csv("BBB_Train.csv", index_col=0)
DDD_Train = pd.read_csv("DDD_Train.csv", index_col=0)
FFF_Train = pd.read_csv("FFF_Train.csv", index_col=0)


C:\Users\Chris\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


# Transforming Data

#### Sort rows of dataframe based on id and date

In [3]:
BBB_Train.head(n=10)

code_module code_presentation  id_student       identifier  date  \
31138         BBB             2013B       23629  BBB 2013B 23629   -23   
31139         BBB             2013B       23629  BBB 2013B 23629   -22   
31140         BBB             2013B       23629  BBB 2013B 23629   -21   
31141         BBB             2013B       23629  BBB 2013B 23629   -20   
31142         BBB             2013B       23629  BBB 2013B 23629   -19   
31143         BBB             2013B       23629  BBB 2013B 23629   -18   
31144         BBB             2013B       23629  BBB 2013B 23629   -17   
31145         BBB             2013B       23629  BBB 2013B 23629   -16   
31146         BBB             2013B       23629  BBB 2013B 23629   -15   
31147         BBB             2013B       23629  BBB 2013B 23629   -14   

       sum_click  asgmts  csum_asgmts  Avg_score final_result  
31138          0       0            0        0.0         Fail  
31139          0       0            0        0.0         Fail  
31140          0       0            0        0.0         Fail  
31141          0       0            0        0.0         Fail  
31142          0       0            0        0.0         Fail  
31143          0       0            0        0.0         Fail  
31144          0       0            0        0.0         Fail  
31145          0       0            0        0.0         Fail  
31146          0       0            0        0.0         Fail  
31147          0       0            0        0.0         Fail

In [4]:
# delete variable presenting the cumulative sum of assignments
BBB_Train = BBB_Train.drop(["csum_asgmts"], axis = 1)
DDD_Train = DDD_Train.drop(["csum_asgmts"], axis = 1)
FFF_Train = FFF_Train.drop(["csum_asgmts"], axis = 1)


In [5]:
# Split the datasets in train and test sets (2014J = test set, while 2013B, 2013J and 2014B comprise the training set)
BBB_Test = BBB_Train[BBB_Train["code_presentation"] == "2014J"]
DDD_Test = DDD_Train[DDD_Train["code_presentation"] == "2014J"]
FFF_Test = FFF_Train[FFF_Train["code_presentation"] == "2014J"]

BBB_Train = BBB_Train[BBB_Train["code_presentation"] != "2014J"]
DDD_Train = DDD_Train[DDD_Train["code_presentation"] != "2014J"]
FFF_Train = FFF_Train[FFF_Train["code_presentation"] != "2014J"]

In [6]:
# sort by identifier and date
BBB_Train = BBB_Train.sort_values(by = ['identifier','date'])
DDD_Train = DDD_Train.sort_values(by = ['identifier','date'])
FFF_Train = FFF_Train.sort_values(by = ['identifier','date'])

BBB_Test = BBB_Test.sort_values(by = ['identifier','date'])
DDD_Test = DDD_Test.sort_values(by = ['identifier','date'])
FFF_Test = FFF_Test.sort_values(by = ['identifier','date'])

In [7]:
BBB_Train.head()

code_module code_presentation  id_student         identifier  date  \
1         BBB             2013B     1008675  BBB 2013B 1008675   -23   
2         BBB             2013B     1008675  BBB 2013B 1008675   -22   
3         BBB             2013B     1008675  BBB 2013B 1008675   -21   
4         BBB             2013B     1008675  BBB 2013B 1008675   -20   
5         BBB             2013B     1008675  BBB 2013B 1008675   -19   

   sum_click  asgmts  Avg_score final_result  
1          0       0        0.0         Pass  
2          0       0        0.0         Pass  
3          0       0        0.0         Pass  
4          0       0        0.0         Pass  
5          0       0        0.0         Pass

In [8]:
BBB_Train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1122969 entries, 1 to 1122969
Data columns (total 9 columns):
code_module          1122969 non-null object
code_presentation    1122969 non-null object
id_student           1122969 non-null int64
identifier           1122969 non-null object
date                 1122969 non-null int64
sum_click            1122969 non-null int64
asgmts               1122969 non-null int64
Avg_score            1122969 non-null float64
final_result         1122969 non-null object
dtypes: float64(1), int64(4), object(4)
memory usage: 85.7+ MB


In [9]:
# Make new datasets with only the relevant variables
BBB_LSTM_Train = BBB_Train.loc[:,["code_presentation","identifier","date","sum_click", "asgmts", "Avg_score", "final_result"]]
BBB_LSTM_Test = BBB_Test.loc[:,["code_presentation","identifier","date","sum_click", "asgmts", "Avg_score", "final_result"]]

DDD_LSTM_Train = DDD_Train.loc[:,["code_presentation","identifier","date","sum_click", "asgmts", "Avg_score", "final_result"]]
DDD_LSTM_Test = DDD_Test.loc[:,["code_presentation","identifier","date","sum_click", "asgmts", "Avg_score", "final_result"]]

FFF_LSTM_Train = FFF_Train.loc[:,["code_presentation","identifier","date","sum_click", "asgmts", "Avg_score", "final_result"]]
FFF_LSTM_Test = FFF_Test.loc[:,["code_presentation","identifier","date","sum_click", "asgmts", "Avg_score", "final_result"]]

In [10]:
BBB_LSTM_Train.head()

code_presentation         identifier  date  sum_click  asgmts  Avg_score  \
1             2013B  BBB 2013B 1008675   -23          0       0        0.0   
2             2013B  BBB 2013B 1008675   -22          0       0        0.0   
3             2013B  BBB 2013B 1008675   -21          0       0        0.0   
4             2013B  BBB 2013B 1008675   -20          0       0        0.0   
5             2013B  BBB 2013B 1008675   -19          0       0        0.0   

  final_result  
1         Pass  
2         Pass  
3         Pass  
4         Pass  
5         Pass

In [11]:
BBB_LSTM_Train.shape

(1122969, 7)

In [12]:
BBB_LSTM_Train.loc[:,["sum_click","asgmts","Avg_score"]][BBB_LSTM_Train["sum_click"]!=-1].shape

(1047040, 3)

#### Calculate percentile-groups based on a student's average daily clicks in a certain decile each semester
Necessary to alleviate effects from structural changes between semesters

In [13]:
# Make a variable called Rank_Clicks which will indicate the rank of a student in terms of clicks on each respective day
# Initial values are set to 0 (or -1 when the student did not sign up to the course yet at that date)

BBB_LSTM_Train["Rank_Clicks"] = 0
DDD_LSTM_Train["Rank_Clicks"] = 0
FFF_LSTM_Train["Rank_Clicks"] = 0

BBB_LSTM_Test["Rank_Clicks"] = 0
DDD_LSTM_Test["Rank_Clicks"] = 0
FFF_LSTM_Test["Rank_Clicks"] = 0

BBB_LSTM_Train["Rank_Clicks"][BBB_LSTM_Train.sum_click == -1] = -1
DDD_LSTM_Train["Rank_Clicks"][DDD_LSTM_Train.sum_click == -1] = -1
FFF_LSTM_Train["Rank_Clicks"][FFF_LSTM_Train.sum_click == -1] = -1

BBB_LSTM_Test["Rank_Clicks"][BBB_LSTM_Test.sum_click == -1] = -1
DDD_LSTM_Test["Rank_Clicks"][DDD_LSTM_Test.sum_click == -1] = -1
FFF_LSTM_Test["Rank_Clicks"][FFF_LSTM_Test.sum_click == -1] = -1

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
C:\Users\Ch

In [14]:

Semesters_Train = BBB_LSTM_Train.code_presentation.unique() # Represents unique course presentations in the respective training set
dfs_Train = list([BBB_LSTM_Train, DDD_LSTM_Train, FFF_LSTM_Train]) #list containing all train datasets
Semesters_Test = BBB_LSTM_Test.code_presentation.unique()
dfs_Test = list([BBB_LSTM_Test, DDD_LSTM_Test, FFF_LSTM_Test])

In [15]:
Percentiles = [0.05, 0.1, 0.15, 0.20, 0.25, 0.30, 0.35, 0.40,0.45,0.50,0.55,0.60,0.65,0.70,0.75,0.80, 0.85,0.90,0.95,1]

In [16]:
percentile_limit = [0]

In [17]:
for i in range(0,len(dfs_Train)):                                   # iterate over the training datasets (different courses)
    for j in range(0,len(Semesters_Train)):                         # iterate over the course presentations within the respective course
        for d in dfs_Train[i]["date"].unique():                     # iterate over the days within the respective presentation of the respective course
            for p in range(0,len(Percentiles)):                     # iterate over the percentile value assigned to the student on that day
                Rowclicks = dfs_Train[i]["sum_click"][dfs_Train[i].sum_click!=-1][dfs_Train[i].date == d][dfs_Train[i].code_presentation == Semesters_Train[j]][dfs_Train[i].sum_click>0]
                                                                    # Rowclicks contains all sum_click values that are larger than 0 for each respective day + presentation + course
                if Rowclicks.empty == False:                        # If rowclicks is not empty add the cut-off value separating the current percentile from the last percentile to the percentile_limit array
                    percentile_limit.append(np.percentile(Rowclicks,Percentiles[p]*100))
                    
                    dfs_Train[i]["Rank_Clicks"][(dfs_Train[i].sum_click != -1) &                                         # The student needs to be registered for the course at the respective day ( not "-1")
                                                (dfs_Train[i].code_presentation == Semesters_Train[j]) &                 
                                                (dfs_Train[i].date == d) &
                                                (dfs_Train[i]["sum_click"] > percentile_limit[p]) &                      # The number of clicks needs to be larger than the cut-off value which separates the previous from the current percentile
                                                (dfs_Train[i]["sum_click"] <= percentile_limit[p+1])] = Percentiles[p]   # The number of clicks needs to be lower or equal than the cut-off value which separates the current from the next percentile
                                                                    
        print("Presentation " + str(j+1) + " out of "+ str(3) + " of course " + str(i+1))
        
    # Intuitively:
    # Students with 0 clicks on the respective day get assigned a "0" in Rank_Clicks
    # The 5% of students with the least amount of clicks on the respective day larger than 0 get assigned a "0.05" in Rank_Clicks
    # The 5% of students with a number of clicks on the respective day higher than the "0.05" group but lower than the top 90% get assigned a "0.10" in Rank_Clicks
    # ...
    # The 5% of students with the highest number of clicks on the respective day get assigned "1.00" in Rank_Clicks


C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Presentation 1 out of 3 of course 1
Presentation 2 out of 3 of course 1
Presentation 3 out of 3 of course 1


C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Presentation 1 out of 3 of course 2
Presentation 2 out of 3 of course 2
Presentation 3 out of 3 of course 2


C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Presentation 1 out of 3 of course 3
Presentation 2 out of 3 of course 3
Presentation 3 out of 3 of course 3


In [18]:
# Same as for the training sets above
for i in range(0,len(dfs_Test)):
    for j in range(0,len(Semesters_Test)):
        for d in dfs_Test[i]["date"].unique():
            for p in range(0,len(Percentiles)):
                Rowclicks = dfs_Test[i]["sum_click"][dfs_Test[i].sum_click!=-1][dfs_Test[i].date == d][dfs_Test[i].code_presentation == Semesters_Test[j]][dfs_Test[i].sum_click>0]
                
                if Rowclicks.empty == False:
                    percentile_limit.append(np.percentile(Rowclicks,Percentiles[p]*100))
                    
                    dfs_Test[i]["Rank_Clicks"][(dfs_Test[i].sum_click != -1) &
                                               (dfs_Test[i].code_presentation == Semesters_Test[j]) & 
                                               (dfs_Test[i].date == d) &
                                               (dfs_Test[i]["sum_click"] > percentile_limit[p]) &
                                               (dfs_Test[i]["sum_click"] <= percentile_limit[p+1])] = Percentiles[p]
                    
                    
        print("Presentation " + str(j+1) + " out of "+ str(1) + " of course " + str(i+1))

C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Presentation 1 out of 4 of course 1


C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Presentation 1 out of 4 of course 2


C:\Users\Chris\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Presentation 1 out of 4 of course 3


In [23]:
# Substitute the "sum_click" feature with the "Rank_Clicks" values
BBB_LSTM_Train["sum_click"] = BBB_LSTM_Train["Rank_Clicks"]
DDD_LSTM_Train["sum_click"] = DDD_LSTM_Train["Rank_Clicks"]
FFF_LSTM_Train["sum_click"] = FFF_LSTM_Train["Rank_Clicks"]

BBB_LSTM_Test["sum_click"] = BBB_LSTM_Test["Rank_Clicks"]
DDD_LSTM_Test["sum_click"] = DDD_LSTM_Test["Rank_Clicks"]
FFF_LSTM_Test["sum_click"] = FFF_LSTM_Test["Rank_Clicks"]

BBB_LSTM_Train = BBB_LSTM_Train.drop('Rank_Clicks',1)
DDD_LSTM_Train = DDD_LSTM_Train.drop('Rank_Clicks',1)
FFF_LSTM_Train = FFF_LSTM_Train.drop('Rank_Clicks',1)

BBB_LSTM_Test = BBB_LSTM_Test.drop('Rank_Clicks',1)
DDD_LSTM_Test = DDD_LSTM_Test.drop('Rank_Clicks',1)
FFF_LSTM_Test = FFF_LSTM_Test.drop('Rank_Clicks',1)

In [24]:
BBB_LSTM_Train.head(n=100)

code_presentation         identifier  date  sum_click  asgmts  Avg_score  \
1               2013B  BBB 2013B 1008675   -23       0.00       0       0.00   
2               2013B  BBB 2013B 1008675   -22       0.00       0       0.00   
3               2013B  BBB 2013B 1008675   -21       0.00       0       0.00   
4               2013B  BBB 2013B 1008675   -20       0.00       0       0.00   
5               2013B  BBB 2013B 1008675   -19       0.00       0       0.00   
6               2013B  BBB 2013B 1008675   -18       0.00       0       0.00   
7               2013B  BBB 2013B 1008675   -17       0.00       0       0.00   
8               2013B  BBB 2013B 1008675   -16       0.00       0       0.00   
9               2013B  BBB 2013B 1008675   -15       0.00       0       0.00   
10              2013B  BBB 2013B 1008675   -14       0.00       0       0.00   
11              2013B  BBB 2013B 1008675   -13       0.00       0       0.00   
12              2013B  BBB 2013B 1008675   -12       0.00       0       0.00   
13              2013B  BBB 2013B 1008675   -11       0.00       0       0.00   
14              2013B  BBB 2013B 1008675   -10       0.00       0       0.00   
15              2013B  BBB 2013B 1008675    -9       0.00       0       0.00   
16              2013B  BBB 2013B 1008675    -8       0.00       0       0.00   
17              2013B  BBB 2013B 1008675    -7       0.00       0       0.00   
18              2013B  BBB 2013B 1008675    -6       0.00       0       0.00   
19              2013B  BBB 2013B 1008675    -5       0.45       0       0.00   
20              2013B  BBB 2013B 1008675    -4       0.40       0       0.00   
21              2013B  BBB 2013B 1008675    -3       0.00       0       0.00   
22              2013B  BBB 2013B 1008675    -2       0.00       0       0.00   
23              2013B  BBB 2013B 1008675    -1       0.10       0       0.00   
24              2013B  BBB 2013B 1008675     0       0.55       0       0.00   
25              2013B  BBB 2013B 1008675     1       0.00       0       0.00   
26              2013B  BBB 2013B 1008675     2       0.90       0       0.00   
27              2013B  BBB 2013B 1008675     3       0.55       0       0.00   
28              2013B  BBB 2013B 1008675     4       0.60       0       0.00   
29              2013B  BBB 2013B 1008675     5       0.10       0       0.00   
30              2013B  BBB 2013B 1008675     6       0.00       0       0.00   
..                ...                ...   ...        ...     ...        ...   
71              2013B  BBB 2013B 1008675    47       0.00       0      65.00   
72              2013B  BBB 2013B 1008675    48       0.10       0      65.00   
73              2013B  BBB 2013B 1008675    49       0.00       0      65.00   
74              2013B  BBB 2013B 1008675    50       0.40       0      65.00   
75              2013B  BBB 2013B 1008675    51       0.15       0      65.00   
76              2013B  BBB 2013B 1008675    52       0.00       0      65.00   
77              2013B  BBB 2013B 1008675    53       0.75       0      65.00   
78              2013B  BBB 2013B 1008675    54       0.45       0      65.00   
79              2013B  BBB 2013B 1008675    55       0.05       0      65.00   
80              2013B  BBB 2013B 1008675    56       0.10       0      65.00   
81              2013B  BBB 2013B 1008675    57       0.00       0      65.00   
82              2013B  BBB 2013B 1008675    58       0.25       0      65.00   
83              2013B  BBB 2013B 1008675    59       0.30       0      65.00   
84              2013B  BBB 2013B 1008675    60       0.35       1      56.67   
85              2013B  BBB 2013B 1008675    61       0.35       0      56.67   
86              2013B  BBB 2013B 1008675    62       0.65       0      56.67   
87              2013B  BBB 2013B 1008675    63       0.20       0      56.67   
88              2013B  BBB 2013B 1008675    64       0.30       0      56.67   
89  

Rescale the "asgmts" and "Avg_score" features to lie between 0 and 1.

asgmts: 0 for lowest number of assignments made in the respective course presentation and 1 for the highest number of assignments made in the course presentation

Avg_score: 0 for the lowest average score in the respective course presentation and 1 for the highest score achieved in the respective course presentation

In [25]:
Semesters_Train = BBB_LSTM_Train.code_presentation.unique()
dfs_Train = list([BBB_LSTM_Train, DDD_LSTM_Train, FFF_LSTM_Train])
Semesters_Test = BBB_LSTM_Test.code_presentation.unique()
dfs_Test = list([BBB_LSTM_Test, DDD_LSTM_Test, FFF_LSTM_Test])

In [26]:
from sklearn.preprocessing import MinMaxScaler

In [27]:
scaler = MinMaxScaler(feature_range=(0,1))

for i in range(0,len(dfs_Train)):
    for j in range(0,len(Semesters_Train)):
        scaler.fit(dfs_Train[i].loc[(dfs_Train[i]["code_presentation"]==Semesters_Train[j]) & (dfs_Train[i]["asgmts"]!=-1), ["asgmts","Avg_score"]])
        dfs_Train[i].loc[(dfs_Train[i]["code_presentation"]==Semesters_Train[j]) & (dfs_Train[i]["asgmts"]!=-1), ["asgmts","Avg_score"]] = scaler.transform(dfs_Train[i].loc[(dfs_Train[i]["code_presentation"]==Semesters_Train[j]) & (dfs_Train[i]["asgmts"]!=-1), ["asgmts","Avg_score"]])


for i in range(0,len(dfs_Test)):
    for j in range(0,len(Semesters_Test)):
        scaler.fit(dfs_Test[i].loc[(dfs_Test[i]["code_presentation"]==Semesters_Test[j])  & (dfs_Test[i]["asgmts"]!=-1), ["asgmts","Avg_score"]])
        dfs_Test[i].loc[(dfs_Test[i]["code_presentation"]==Semesters_Test[j]) & (dfs_Test[i]["asgmts"]!=-1), ["asgmts","Avg_score"]] = scaler.transform(dfs_Test[i].loc[(dfs_Test[i]["code_presentation"]==Semesters_Test[j])  & (dfs_Test[i]["asgmts"]!=-1), ["asgmts","Avg_score"]])

In [28]:
BBB_Train.head(n=100)

code_module code_presentation  id_student         identifier  date  \
1           BBB             2013B     1008675  BBB 2013B 1008675   -23   
2           BBB             2013B     1008675  BBB 2013B 1008675   -22   
3           BBB             2013B     1008675  BBB 2013B 1008675   -21   
4           BBB             2013B     1008675  BBB 2013B 1008675   -20   
5           BBB             2013B     1008675  BBB 2013B 1008675   -19   
6           BBB             2013B     1008675  BBB 2013B 1008675   -18   
7           BBB             2013B     1008675  BBB 2013B 1008675   -17   
8           BBB             2013B     1008675  BBB 2013B 1008675   -16   
9           BBB             2013B     1008675  BBB 2013B 1008675   -15   
10          BBB             2013B     1008675  BBB 2013B 1008675   -14   
11          BBB             2013B     1008675  BBB 2013B 1008675   -13   
12          BBB             2013B     1008675  BBB 2013B 1008675   -12   
13          BBB             2013B     1008675  BBB 2013B 1008675   -11   
14          BBB             2013B     1008675  BBB 2013B 1008675   -10   
15          BBB             2013B     1008675  BBB 2013B 1008675    -9   
16          BBB             2013B     1008675  BBB 2013B 1008675    -8   
17          BBB             2013B     1008675  BBB 2013B 1008675    -7   
18          BBB             2013B     1008675  BBB 2013B 1008675    -6   
19          BBB             2013B     1008675  BBB 2013B 1008675    -5   
20          BBB             2013B     1008675  BBB 2013B 1008675    -4   
21          BBB             2013B     1008675  BBB 2013B 1008675    -3   
22          BBB             2013B     1008675  BBB 2013B 1008675    -2   
23          BBB             2013B     1008675  BBB 2013B 1008675    -1   
24          BBB             2013B     1008675  BBB 2013B 1008675     0   
25          BBB             2013B     1008675  BBB 2013B 1008675     1   
26          BBB             2013B     1008675  BBB 2013B 1008675     2   
27          BBB             2013B     1008675  BBB 2013B 1008675     3   
28          BBB             2013B     1008675  BBB 2013B 1008675     4   
29          BBB             2013B     1008675  BBB 2013B 1008675     5   
30          BBB             2013B     1008675  BBB 2013B 1008675     6   
..          ...               ...         ...                ...   ...   
71          BBB             2013B     1008675  BBB 2013B 1008675    47   
72          BBB             2013B     1008675  BBB 2013B 1008675    48   
73          BBB             2013B     1008675  BBB 2013B 1008675    49   
74          BBB             2013B     1008675  BBB 2013B 1008675    50   
75          BBB             2013B     1008675  BBB 2013B 1008675    51   
76          BBB             2013B     1008675  BBB 2013B 1008675    52   
77          BBB             2013B     1008675  BBB 2013B 1008675    53   
78          BBB             2013B     1008675  BBB 2013B 1008675    54   
79          BBB             2013B     1008675  BBB 2013B 1008675    55   
80          BBB             2013B     1008675  BBB 2013B 1008675    56   
81          BBB             2013B     1008675  BBB 2013B 1008675    57   
82          BBB             2013B     1008675  BBB 2013B 1008675    58   
83          BBB             2013B     1008675  BBB 2013B 1008675    59   
84          BBB             2013B     1008675  BBB 2013B 1008675    60   
85          BBB             2013B     1008675  BBB 2013B 1008675    61   
86          BBB             2013B     1008675  BBB 2013B 1008675    62   
87          BBB             2013B     1008675  BBB 2013B 1008675    63   
88          BBB             2013B     1008675  BBB 2013B 1008675    64   
89          BBB             2013B     1008675  BBB 2013B 1008675    65   
90          BBB             2013B     1008675  BBB 2013B 1008675    66   
91          BBB             2013B     1008675  BBB 2013B 1008675    67   
92          BBB             2013B     1008675  BBB 2013B 1008675    68   
93      

In [34]:
# Export the transformed datasets to .csv files for future reference (required uncommenting this block)

#BBB_LSTM_Train.to_csv("Normalized_BBB_Train.csv")
#DDD_LSTM_Train.to_csv("Normalized_DDD_Train.csv")
#FFF_LSTM_Train.to_csv("Normalized_FFF_Train.csv")

#BBB_LSTM_Test.to_csv("Normalized_BBB_Test.csv")
#DDD_LSTM_Test.to_csv("Normalized_DDD_Test.csv")
#FFF_LSTM_Test.to_csv("Normalized_FFF_Test.csv")

In [13]:
# Load the previously created .csv files

BBB_LSTM_Train = pd.read_csv("Normalized_BBB_Train.csv", index_col=0)
DDD_LSTM_Train = pd.read_csv("Normalized_DDD_Train.csv", index_col=0)
FFF_LSTM_Train = pd.read_csv("Normalized_FFF_Train.csv", index_col=0)

BBB_LSTM_Test = pd.read_csv("Normalized_BBB_Test.csv", index_col=0)
DDD_LSTM_Test = pd.read_csv("Normalized_DDD_Test.csv", index_col=0)
FFF_LSTM_Test = pd.read_csv("Normalized_FFF_Test.csv", index_col=0)

C:\Users\Chris\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [14]:
BBB_LSTM_Train.shape

(1122969, 7)

In [15]:
BBB_LSTM_Train.head(n=100)

code_presentation         identifier  date  sum_click    asgmts  \
1               2013B  BBB 2013B 1008675   -23       0.00  0.000000   
2               2013B  BBB 2013B 1008675   -22       0.00  0.000000   
3               2013B  BBB 2013B 1008675   -21       0.00  0.000000   
4               2013B  BBB 2013B 1008675   -20       0.00  0.000000   
5               2013B  BBB 2013B 1008675   -19       0.00  0.000000   
6               2013B  BBB 2013B 1008675   -18       0.00  0.000000   
7               2013B  BBB 2013B 1008675   -17       0.00  0.000000   
8               2013B  BBB 2013B 1008675   -16       0.00  0.000000   
9               2013B  BBB 2013B 1008675   -15       0.00  0.000000   
10              2013B  BBB 2013B 1008675   -14       0.00  0.000000   
11              2013B  BBB 2013B 1008675   -13       0.00  0.000000   
12              2013B  BBB 2013B 1008675   -12       0.00  0.000000   
13              2013B  BBB 2013B 1008675   -11       0.00  0.000000   
14              2013B  BBB 2013B 1008675   -10       0.00  0.000000   
15              2013B  BBB 2013B 1008675    -9       0.00  0.000000   
16              2013B  BBB 2013B 1008675    -8       0.00  0.000000   
17              2013B  BBB 2013B 1008675    -7       0.00  0.000000   
18              2013B  BBB 2013B 1008675    -6       0.00  0.000000   
19              2013B  BBB 2013B 1008675    -5       0.45  0.000000   
20              2013B  BBB 2013B 1008675    -4       0.40  0.000000   
21              2013B  BBB 2013B 1008675    -3       0.00  0.000000   
22              2013B  BBB 2013B 1008675    -2       0.00  0.000000   
23              2013B  BBB 2013B 1008675    -1       0.10  0.000000   
24              2013B  BBB 2013B 1008675     0       0.55  0.000000   
25              2013B  BBB 2013B 1008675     1       0.00  0.000000   
26              2013B  BBB 2013B 1008675     2       0.90  0.000000   
27              2013B  BBB 2013B 1008675     3       0.55  0.000000   
28              2013B  BBB 2013B 1008675     4       0.60  0.000000   
29              2013B  BBB 2013B 1008675     5       0.10  0.000000   
30              2013B  BBB 2013B 1008675     6       0.00  0.000000   
..                ...                ...   ...        ...       ...   
71              2013B  BBB 2013B 1008675    47       0.00  0.000000   
72              2013B  BBB 2013B 1008675    48       0.10  0.000000   
73              2013B  BBB 2013B 1008675    49       0.00  0.000000   
74              2013B  BBB 2013B 1008675    50       0.40  0.000000   
75              2013B  BBB 2013B 1008675    51       0.15  0.000000   
76              2013B  BBB 2013B 1008675    52       0.00  0.000000   
77              2013B  BBB 2013B 1008675    53       0.75  0.000000   
78              2013B  BBB 2013B 1008675    54       0.45  0.000000   
79              2013B  BBB 2013B 1008675    55       0.05  0.000000   
80              2013B  BBB 2013B 1008675    56       0.10  0.000000   
81              2013B  BBB 2013B 1008675    57       0.00  0.000000   
82              2013B  BBB 2013B 1008675    58       0.25  0.000000   
83              2013B  BBB 2013B 1008675    59       0.30  0.000000   
84              2013B  BBB 2013B 1008675    60       0.35  0.142857   
85              2013B  BBB 2013B 1008675    61       0.35  0.000000   
86              2013B  BBB 2013B 1008675    62       0.65  0.000000   
87              2013B  BBB 2013B 1008675    63       0.20  0.000000   
88              2013B  BBB 2013B 1008675    64       0.30  0.000000   
89              2013B  BBB 2013B 1008675    65       0.25  0.000000   
90              2013B  BBB 2013B 1008675    66       0.00  0.000000   
91              2013B  BBB 2013B 1008675    67       0.40  0.000000   
92              2013B  BBB 2013B 1008675    68       0.50  0.000000   
93              2013B  BBB 2013B 1008675    69       0.50  0.000000   
94              2013B  BBB 2013B 1008675    70       0.45  0.000000   
95              2013B  BBB 2

# Reformat data

In [16]:
BBB_AUX_Train = pd.concat((BBB_Train.loc[:,"identifier"],BBB_Train.loc[:,"final_result"]), axis = 1)
BBB_AUX_Test = pd.concat((BBB_Test.loc[:,"identifier"],BBB_Test.loc[:,"final_result"]), axis = 1)

DDD_AUX_Train = pd.concat((DDD_Train.loc[:,"identifier"],DDD_Train.loc[:,"final_result"]), axis = 1)
DDD_AUX_Test = pd.concat((DDD_Test.loc[:,"identifier"],DDD_Test.loc[:,"final_result"]), axis = 1)

FFF_AUX_Train = pd.concat((FFF_Train.loc[:,"identifier"],FFF_Train.loc[:,"final_result"]), axis = 1)
FFF_AUX_Test = pd.concat((FFF_Test.loc[:,"identifier"],FFF_Test.loc[:,"final_result"]), axis = 1)

In [17]:
# Create an array with the target variable (final_result) for each student
BBB_AUX_Train = BBB_AUX_Train.drop_duplicates()
BBB_AUX_Test = BBB_AUX_Test.drop_duplicates()

DDD_AUX_Train = DDD_AUX_Train.drop_duplicates()
DDD_AUX_Test = DDD_AUX_Test.drop_duplicates()

FFF_AUX_Train = FFF_AUX_Train.drop_duplicates()
FFF_AUX_Test = FFF_AUX_Test.drop_duplicates()

In [18]:
FFF_AUX_Train.shape

(3819, 2)

In [19]:
BBB_AUX_Train

identifier final_result
1        BBB 2013B 1008675         Pass
292       BBB 2013B 104054  Distinction
583       BBB 2013B 108377         Pass
874       BBB 2013B 108589         Pass
1165     BBB 2013B 1098728         Fail
1456      BBB 2013B 109938  Distinction
1747      BBB 2013B 110881         Pass
2038      BBB 2013B 116603  Distinction
2329      BBB 2013B 117071         Fail
2620      BBB 2013B 120994         Fail
2911      BBB 2013B 123860         Pass
3202      BBB 2013B 126322         Pass
3493      BBB 2013B 126411         Fail
3784      BBB 2013B 133155         Pass
4075      BBB 2013B 133748         Fail
4366      BBB 2013B 142324         Fail
4657     BBB 2013B 1436591  Distinction
4948      BBB 2013B 143741  Distinction
5239      BBB 2013B 145544         Pass
5530      BBB 2013B 147876         Fail
5821     BBB 2013B 1480800         Pass
6112     BBB 2013B 1493476         Pass
6403      BBB 2013B 151697         Pass
6694     BBB 2013B 1521540  Distinction
6985      BBB 2013B 152910         Pass
7276      BBB 2013B 152929         Pass
7567      BBB 2013B 153212  Distinction
7858      BBB 2013B 154027         Pass
8149      BBB 2013B 154125         Fail
8440      BBB 2013B 154540         Pass
...                    ...          ...
1114240   BBB 2014B 635136         Pass
1114531   BBB 2014B 635173  Distinction
1114822   BBB 2014B 635228         Pass
1115113   BBB 2014B 635230  Distinction
1115404   BBB 2014B 635241         Pass
1115695   BBB 2014B 635244         Pass
1115986   BBB 2014B 635246         Fail
1116277   BBB 2014B 635247         Pass
1116568   BBB 2014B 635432         Fail
1116859   BBB 2014B 635433         Pass
1117150   BBB 2014B 635435         Pass
1117441   BBB 2014B 635473         Fail
1117732   BBB 2014B 635591         Pass
1118023   BBB 2014B 635755  Distinction
1118314   BBB 2014B 635761         Pass
1118605   BBB 2014B 635764         Pass
1118896   BBB 2014B 635771         Pass
1119187   BBB 2014B 635773         Pass
1119478   BBB 2014B 635774         Pass
1119769   BBB 2014B 635788         Pass
1120060    BBB 2014B 63721  Distinction
1120351    BBB 2014B 64275         Pass
1120642    BBB 2014B 72400  Distinction
1120933    BBB 2014B 78255         Fail
1121224    BBB 2014B 89092         Pass
1121515    BBB 2014B 89778  Distinction
1121806    BBB 2014B 90361         Pass
1122097    BBB 2014B 92341         Fail
1122388    BBB 2014B 97006         Pass
1122679    BBB 2014B 99799         Pass

[3859 rows x 2 columns]

Create 3D arrays with the dimensions representing [samples, sequences, features] = [students, days, (clicks/asgmts/avg_score)]

In [20]:
BBB_LSTM_Train_List = BBB_LSTM_Train['identifier'].unique().tolist()
BBB_LSTM_Test_List = BBB_LSTM_Test['identifier'].unique().tolist()

DDD_LSTM_Train_List = DDD_LSTM_Train['identifier'].unique().tolist()
DDD_LSTM_Test_List = DDD_LSTM_Test['identifier'].unique().tolist()

FFF_LSTM_Train_List = FFF_LSTM_Train['identifier'].unique().tolist()
FFF_LSTM_Test_List = FFF_LSTM_Test['identifier'].unique().tolist()

In [21]:
BBB_LSTM_Train_3D = np.asarray(list(BBB_LSTM_Train.groupby('identifier').apply(pd.DataFrame.as_matrix))) 
BBB_LSTM_Test_3D = np.asarray(list(BBB_LSTM_Test.groupby('identifier').apply(pd.DataFrame.as_matrix)))

DDD_LSTM_Train_3D = np.asarray(list(DDD_LSTM_Train.groupby('identifier').apply(pd.DataFrame.as_matrix))) 
DDD_LSTM_Test_3D = np.asarray(list(DDD_LSTM_Test.groupby('identifier').apply(pd.DataFrame.as_matrix)))

FFF_LSTM_Train_3D = np.asarray(list(FFF_LSTM_Train.groupby('identifier').apply(pd.DataFrame.as_matrix))) 
FFF_LSTM_Test_3D = np.asarray(list(FFF_LSTM_Test.groupby('identifier').apply(pd.DataFrame.as_matrix)))

In [22]:
BBB_LSTM_Train_3D

array([[['2013B', 'BBB 2013B 1008675', -23, ..., 0.0, 0.0, 'Pass'],
        ['2013B', 'BBB 2013B 1008675', -22, ..., 0.0, 0.0, 'Pass'],
        ['2013B', 'BBB 2013B 1008675', -21, ..., 0.0, 0.0, 'Pass'],
        ..., 
        ['2013B', 'BBB 2013B 1008675', 265, ..., -1.0, -1.0, 'Pass'],
        ['2013B', 'BBB 2013B 1008675', 266, ..., -1.0, -1.0, 'Pass'],
        ['2013B', 'BBB 2013B 1008675', 267, ..., -1.0, -1.0, 'Pass']],

       [['2013B', 'BBB 2013B 104054', -23, ..., -1.0, -1.0, 'Distinction'],
        ['2013B', 'BBB 2013B 104054', -22, ..., -1.0, -1.0, 'Distinction'],
        ['2013B', 'BBB 2013B 104054', -21, ..., -1.0, -1.0, 'Distinction'],
        ..., 
        ['2013B', 'BBB 2013B 104054', 265, ..., -1.0, -1.0, 'Distinction'],
        ['2013B', 'BBB 2013B 104054', 266, ..., -1.0, -1.0, 'Distinction'],
        ['2013B', 'BBB 2013B 104054', 267, ..., -1.0, -1.0, 'Distinction']],

       [['2013B', 'BBB 2013B 108377', -23, ..., 0.0, 0.0, 'Pass'],
        ['2013B', 'BBB 2013B 10

In [23]:
# Remove all unnecessary features, only the clicks, asgmts and avg score are necessary
BBB_Train_3D = np.delete(BBB_LSTM_Train_3D,[0,1,2,6],axis=2)
BBB_Test_3D = np.delete(BBB_LSTM_Test_3D,[0,1,2,6],axis=2)

DDD_Train_3D = np.delete(DDD_LSTM_Train_3D,[0,1,2,6],axis=2)
DDD_Test_3D = np.delete(DDD_LSTM_Test_3D,[0,1,2,6],axis=2)

FFF_Train_3D = np.delete(FFF_LSTM_Train_3D,[0,1,2,6],axis=2)
FFF_Test_3D = np.delete(FFF_LSTM_Test_3D,[0,1,2,6],axis=2)

In [24]:
BBB_Train_3D

array([[[0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        ..., 
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0]],

       [[-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0],
        ..., 
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0]],

       [[0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        ..., 
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0]],

       ..., 
       [[0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        ..., 
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0]],

       [[0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        ..., 
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0],
        [-1.0, -1.0, -1.0]],

       [[0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        [0.0, 0.0, 0.0],
        ..., 
        [-1.0,

Array with target variables for each student

In [25]:
BBB_Train_Targets = BBB_AUX_Train.iloc[:,-1]
DDD_Train_Targets = DDD_AUX_Train.iloc[:,-1]
FFF_Train_Targets = FFF_AUX_Train.iloc[:,-1]

BBB_Test_Targets = BBB_AUX_Test.iloc[:,-1]
DDD_Test_Targets = DDD_AUX_Test.iloc[:,-1]
FFF_Test_Targets = FFF_AUX_Test.iloc[:,-1]

In [26]:
BBB_AUX_Train = BBB_AUX_Train.iloc[:,1:30]
DDD_AUX_Train = DDD_AUX_Train.iloc[:,1:30]
FFF_AUX_Train = FFF_AUX_Train.iloc[:,1:30]

BBB_AUX_Test = BBB_AUX_Test.iloc[:,1:30]
DDD_AUX_Test = DDD_AUX_Test.iloc[:,1:30]
FFF_AUX_Test = FFF_AUX_Test.iloc[:,1:30]

In [27]:
DDD_Train_Targets

1565581           Fail
1565868    Distinction
1566155           Pass
1566442           Pass
1566729           Pass
1567016           Fail
1567303           Fail
1567590           Fail
1567877           Pass
1568164           Fail
1568451           Fail
1568738           Fail
1569025           Fail
1569312           Fail
1569599           Pass
1569886           Pass
1570173    Distinction
1570460           Pass
1570747           Fail
1571034           Pass
1571321           Pass
1571608           Pass
1571895           Fail
1572182           Fail
1572469           Pass
1572756           Pass
1573043           Fail
1573330           Fail
1573617           Fail
1573904           Fail
              ...     
2369468           Pass
2369755           Pass
2370042           Fail
2370329           Pass
2370616    Distinction
2370903           Fail
2371190           Pass
2371477           Pass
2371764           Pass
2372051    Distinction
2372338           Pass
2372625    Distinction
2372912    

In [28]:
BBB_Train_Targets_1D = np.array(BBB_Train_Targets)
DDD_Train_Targets_1D = np.array(DDD_Train_Targets)
FFF_Train_Targets_1D = np.array(FFF_Train_Targets)

BBB_Test_Targets_1D = np.array(BBB_Test_Targets)
DDD_Test_Targets_1D = np.array(DDD_Test_Targets)
FFF_Test_Targets_1D = np.array(FFF_Test_Targets)


In [29]:
# Relabel the target classes
BBB_Train_Targets_1D[BBB_Train_Targets_1D == "Fail"] = 0
BBB_Train_Targets_1D[BBB_Train_Targets_1D == "Pass"] = 1
BBB_Train_Targets_1D[BBB_Train_Targets_1D == "Distinction"] = 1

DDD_Train_Targets_1D[DDD_Train_Targets_1D == "Fail"] = 0
DDD_Train_Targets_1D[DDD_Train_Targets_1D == "Pass"] = 1
DDD_Train_Targets_1D[DDD_Train_Targets_1D == "Distinction"] = 1

FFF_Train_Targets_1D[FFF_Train_Targets_1D == "Fail"] = 0
FFF_Train_Targets_1D[FFF_Train_Targets_1D == "Pass"] = 1
FFF_Train_Targets_1D[FFF_Train_Targets_1D == "Distinction"] = 1

BBB_Test_Targets_1D[BBB_Test_Targets_1D == "Fail"] = 0
BBB_Test_Targets_1D[BBB_Test_Targets_1D == "Pass"] = 1
BBB_Test_Targets_1D[BBB_Test_Targets_1D == "Distinction"] = 1

DDD_Test_Targets_1D[DDD_Test_Targets_1D == "Fail"] = 0
DDD_Test_Targets_1D[DDD_Test_Targets_1D == "Pass"] = 1
DDD_Test_Targets_1D[DDD_Test_Targets_1D == "Distinction"] = 1

FFF_Test_Targets_1D[FFF_Test_Targets_1D == "Fail"] = 0
FFF_Test_Targets_1D[FFF_Test_Targets_1D == "Pass"] = 1
FFF_Test_Targets_1D[FFF_Test_Targets_1D == "Distinction"] = 1

In [30]:
BBB_Train_Targets_1D = BBB_Train_Targets_1D.astype('int64')
DDD_Train_Targets_1D = DDD_Train_Targets_1D.astype('int64')
FFF_Train_Targets_1D = FFF_Train_Targets_1D.astype('int64')

BBB_Test_Targets_1D = BBB_Test_Targets_1D.astype('int64')
DDD_Test_Targets_1D = DDD_Test_Targets_1D.astype('int64')
FFF_Test_Targets_1D = FFF_Test_Targets_1D.astype('int64')

In [31]:
print(FFF_Train_Targets_1D.shape)

(3819,)


In [32]:
print(FFF_Train_3D.shape)

(3819, 259, 3)


In [33]:
print(FFF_AUX_Train.shape)

(3819, 1)


#### Exclude timestep(s) in sequences to be able to build the models for multiple points in time

In [34]:
Precourse_days_BBB = abs(BBB_Train["date"].min()) # Take the maximum number of days a student made use of the VLE before the course started
Precourse_days_DDD = abs(DDD_Train["date"].min())
Precourse_days_FFF = abs(FFF_Train["date"].min())

cut_off_BBB = int(round((BBB_Train_3D.shape[1] - Precourse_days_BBB)/10)) # Compute the cut-off dates for the deciles required to separate the course duration in 10 "equal" timesteps at which the models can perform predictions
cut_off_DDD = int(round((DDD_Train_3D.shape[1] - Precourse_days_DDD)/10))
cut_off_FFF = int(round((FFF_Train_3D.shape[1] - Precourse_days_FFF)/10))

In [35]:
# Create the datasets corresponding the each decile for the respective course
BBB_Train_Decile0 = BBB_Train_3D[:,0:Precourse_days_BBB,:]
BBB_Train_Decile1 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB),:]
BBB_Train_Decile2 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*2),:]
BBB_Train_Decile3 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*3),:]
BBB_Train_Decile4 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*4),:]
BBB_Train_Decile5 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*5),:]
BBB_Train_Decile6 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*6),:]
BBB_Train_Decile7 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*7),:]
BBB_Train_Decile8 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*8),:]
BBB_Train_Decile9 = BBB_Train_3D[:,0:(Precourse_days_BBB+cut_off_BBB*9),:]
BBB_Train_Decile10 = BBB_Train_3D

BBB_Test_Decile0 = BBB_Test_3D[:,0:Precourse_days_BBB,:]
BBB_Test_Decile1 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB),:]
BBB_Test_Decile2 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*2),:]
BBB_Test_Decile3 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*3),:]
BBB_Test_Decile4 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*4),:]
BBB_Test_Decile5 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*5),:]
BBB_Test_Decile6 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*6),:]
BBB_Test_Decile7 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*7),:]
BBB_Test_Decile8 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*8),:]
BBB_Test_Decile9 = BBB_Test_3D[:,0:(Precourse_days_BBB+cut_off_BBB*9),:]
BBB_Test_Decile10 = BBB_Test_3D

In [36]:
DDD_Train_Decile0 = DDD_Train_3D[:,0:Precourse_days_DDD,:]
DDD_Train_Decile1 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD),:]
DDD_Train_Decile2 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*2),:]
DDD_Train_Decile3 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*3),:]
DDD_Train_Decile4 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*4),:]
DDD_Train_Decile5 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*5),:]
DDD_Train_Decile6 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*6),:]
DDD_Train_Decile7 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*7),:]
DDD_Train_Decile8 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*8),:]
DDD_Train_Decile9 = DDD_Train_3D[:,0:(Precourse_days_DDD+cut_off_DDD*9),:]
DDD_Train_Decile10 = DDD_Train_3D

DDD_Test_Decile0 = DDD_Test_3D[:,0:Precourse_days_DDD,:]
DDD_Test_Decile1 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD),:]
DDD_Test_Decile2 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*2),:]
DDD_Test_Decile3 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*3),:]
DDD_Test_Decile4 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*4),:]
DDD_Test_Decile5 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*5),:]
DDD_Test_Decile6 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*6),:]
DDD_Test_Decile7 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*7),:]
DDD_Test_Decile8 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*8),:]
DDD_Test_Decile9 = DDD_Test_3D[:,0:(Precourse_days_DDD+cut_off_DDD*9),:]
DDD_Test_Decile10 = DDD_Test_3D

In [37]:
FFF_Train_Decile0 = FFF_Train_3D[:,0:Precourse_days_FFF,:]
FFF_Train_Decile1 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF),:]
FFF_Train_Decile2 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*2),:]
FFF_Train_Decile3 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*3),:]
FFF_Train_Decile4 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*4),:]
FFF_Train_Decile5 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*5),:]
FFF_Train_Decile6 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*6),:]
FFF_Train_Decile7 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*7),:]
FFF_Train_Decile8 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*8),:]
FFF_Train_Decile9 = FFF_Train_3D[:,0:(Precourse_days_FFF+cut_off_FFF*9),:]
FFF_Train_Decile10 = FFF_Train_3D

FFF_Test_Decile0 = FFF_Test_3D[:,0:Precourse_days_FFF,:]
FFF_Test_Decile1 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF),:]
FFF_Test_Decile2 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*2),:]
FFF_Test_Decile3 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*3),:]
FFF_Test_Decile4 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*4),:]
FFF_Test_Decile5 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*5),:]
FFF_Test_Decile6 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*6),:]
FFF_Test_Decile7 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*7),:]
FFF_Test_Decile8 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*8),:]
FFF_Test_Decile9 = FFF_Test_3D[:,0:(Precourse_days_FFF+cut_off_FFF*9),:]
FFF_Test_Decile10 = FFF_Test_3D

# Export arrays to .pkl files

In [64]:
pd.to_pickle(BBB_Train_Decile0, "BBB_Train_Decile0.pkl")
pd.to_pickle(BBB_Train_Decile1, "BBB_Train_Decile1.pkl")
pd.to_pickle(BBB_Train_Decile2, "BBB_Train_Decile2.pkl")
pd.to_pickle(BBB_Train_Decile3, "BBB_Train_Decile3.pkl")
pd.to_pickle(BBB_Train_Decile4, "BBB_Train_Decile4.pkl")
pd.to_pickle(BBB_Train_Decile5, "BBB_Train_Decile5.pkl")
pd.to_pickle(BBB_Train_Decile6, "BBB_Train_Decile6.pkl")
pd.to_pickle(BBB_Train_Decile7, "BBB_Train_Decile7.pkl")
pd.to_pickle(BBB_Train_Decile8, "BBB_Train_Decile8.pkl")
pd.to_pickle(BBB_Train_Decile9, "BBB_Train_Decile9.pkl")
pd.to_pickle(BBB_Train_Decile10, "BBB_Train_Decile10.pkl")

pd.to_pickle(BBB_Test_Decile0, "BBB_Test_Decile0.pkl")
pd.to_pickle(BBB_Test_Decile1, "BBB_Test_Decile1.pkl")
pd.to_pickle(BBB_Test_Decile2, "BBB_Test_Decile2.pkl")
pd.to_pickle(BBB_Test_Decile3, "BBB_Test_Decile3.pkl")
pd.to_pickle(BBB_Test_Decile4, "BBB_Test_Decile4.pkl")
pd.to_pickle(BBB_Test_Decile5, "BBB_Test_Decile5.pkl")
pd.to_pickle(BBB_Test_Decile6, "BBB_Test_Decile6.pkl")
pd.to_pickle(BBB_Test_Decile7, "BBB_Test_Decile7.pkl")
pd.to_pickle(BBB_Test_Decile8, "BBB_Test_Decile8.pkl")
pd.to_pickle(BBB_Test_Decile9, "BBB_Test_Decile9.pkl")
pd.to_pickle(BBB_Test_Decile10, "BBB_Test_Decile10.pkl")

pd.to_pickle(BBB_Train_Targets_1D, "BBB_Train_Targets_1D.pkl")
pd.to_pickle(BBB_Test_Targets_1D, "BBB_Test_Targets_1D.pkl")

In [65]:
pd.to_pickle(DDD_Train_Decile0, "DDD_Train_Decile0.pkl")
pd.to_pickle(DDD_Train_Decile1, "DDD_Train_Decile1.pkl")
pd.to_pickle(DDD_Train_Decile2, "DDD_Train_Decile2.pkl")
pd.to_pickle(DDD_Train_Decile3, "DDD_Train_Decile3.pkl")
pd.to_pickle(DDD_Train_Decile4, "DDD_Train_Decile4.pkl")
pd.to_pickle(DDD_Train_Decile5, "DDD_Train_Decile5.pkl")
pd.to_pickle(DDD_Train_Decile6, "DDD_Train_Decile6.pkl")
pd.to_pickle(DDD_Train_Decile7, "DDD_Train_Decile7.pkl")
pd.to_pickle(DDD_Train_Decile8, "DDD_Train_Decile8.pkl")
pd.to_pickle(DDD_Train_Decile9, "DDD_Train_Decile9.pkl")
pd.to_pickle(DDD_Train_Decile10, "DDD_Train_Decile10.pkl")

pd.to_pickle(DDD_Test_Decile0, "DDD_Test_Decile0.pkl")
pd.to_pickle(DDD_Test_Decile1, "DDD_Test_Decile1.pkl")
pd.to_pickle(DDD_Test_Decile2, "DDD_Test_Decile2.pkl")
pd.to_pickle(DDD_Test_Decile3, "DDD_Test_Decile3.pkl")
pd.to_pickle(DDD_Test_Decile4, "DDD_Test_Decile4.pkl")
pd.to_pickle(DDD_Test_Decile5, "DDD_Test_Decile5.pkl")
pd.to_pickle(DDD_Test_Decile6, "DDD_Test_Decile6.pkl")
pd.to_pickle(DDD_Test_Decile7, "DDD_Test_Decile7.pkl")
pd.to_pickle(DDD_Test_Decile8, "DDD_Test_Decile8.pkl")
pd.to_pickle(DDD_Test_Decile9, "DDD_Test_Decile9.pkl")
pd.to_pickle(DDD_Test_Decile10, "DDD_Test_Decile10.pkl")

pd.to_pickle(DDD_Train_Targets_1D, "DDD_Train_Targets_1D.pkl")
pd.to_pickle(DDD_Test_Targets_1D, "DDD_Test_Targets_1D.pkl")

In [66]:
pd.to_pickle(FFF_Train_Decile0, "FFF_Train_Decile0.pkl")
pd.to_pickle(FFF_Train_Decile1, "FFF_Train_Decile1.pkl")
pd.to_pickle(FFF_Train_Decile2, "FFF_Train_Decile2.pkl")
pd.to_pickle(FFF_Train_Decile3, "FFF_Train_Decile3.pkl")
pd.to_pickle(FFF_Train_Decile4, "FFF_Train_Decile4.pkl")
pd.to_pickle(FFF_Train_Decile5, "FFF_Train_Decile5.pkl")
pd.to_pickle(FFF_Train_Decile6, "FFF_Train_Decile6.pkl")
pd.to_pickle(FFF_Train_Decile7, "FFF_Train_Decile7.pkl")
pd.to_pickle(FFF_Train_Decile8, "FFF_Train_Decile8.pkl")
pd.to_pickle(FFF_Train_Decile9, "FFF_Train_Decile9.pkl")
pd.to_pickle(FFF_Train_Decile10, "FFF_Train_Decile10.pkl")

pd.to_pickle(FFF_Test_Decile0, "FFF_Test_Decile0.pkl")
pd.to_pickle(FFF_Test_Decile1, "FFF_Test_Decile1.pkl")
pd.to_pickle(FFF_Test_Decile2, "FFF_Test_Decile2.pkl")
pd.to_pickle(FFF_Test_Decile3, "FFF_Test_Decile3.pkl")
pd.to_pickle(FFF_Test_Decile4, "FFF_Test_Decile4.pkl")
pd.to_pickle(FFF_Test_Decile5, "FFF_Test_Decile5.pkl")
pd.to_pickle(FFF_Test_Decile6, "FFF_Test_Decile6.pkl")
pd.to_pickle(FFF_Test_Decile7, "FFF_Test_Decile7.pkl")
pd.to_pickle(FFF_Test_Decile8, "FFF_Test_Decile8.pkl")
pd.to_pickle(FFF_Test_Decile9, "FFF_Test_Decile9.pkl")
pd.to_pickle(FFF_Test_Decile10, "FFF_Test_Decile10.pkl")

pd.to_pickle(FFF_Train_Targets_1D, "FFF_Train_Targets_1D.pkl")
pd.to_pickle(FFF_Test_Targets_1D, "FFF_Test_Targets_1D.pkl")

In [2]:
BBB_Train_Decile0 = pd.read_pickle("BBB_Train_Decile0.pkl")
BBB_Train_Decile1 = pd.read_pickle("BBB_Train_Decile1.pkl")
BBB_Train_Decile2 = pd.read_pickle("BBB_Train_Decile2.pkl")
BBB_Train_Decile3 = pd.read_pickle("BBB_Train_Decile3.pkl")
BBB_Train_Decile4 = pd.read_pickle("BBB_Train_Decile4.pkl")
BBB_Train_Decile5 = pd.read_pickle("BBB_Train_Decile5.pkl")
BBB_Train_Decile6 = pd.read_pickle("BBB_Train_Decile6.pkl")
BBB_Train_Decile7 = pd.read_pickle("BBB_Train_Decile7.pkl")
BBB_Train_Decile8 = pd.read_pickle("BBB_Train_Decile8.pkl")
BBB_Train_Decile9 = pd.read_pickle("BBB_Train_Decile9.pkl")
BBB_Train_Decile10 = pd.read_pickle("BBB_Train_Decile10.pkl")

BBB_Test_Decile0 = pd.read_pickle("BBB_Test_Decile0.pkl")
BBB_Test_Decile1 = pd.read_pickle("BBB_Test_Decile1.pkl")
BBB_Test_Decile2 = pd.read_pickle("BBB_Test_Decile2.pkl")
BBB_Test_Decile3 = pd.read_pickle("BBB_Test_Decile3.pkl")
BBB_Test_Decile4 = pd.read_pickle("BBB_Test_Decile4.pkl")
BBB_Test_Decile5 = pd.read_pickle("BBB_Test_Decile5.pkl")
BBB_Test_Decile6 = pd.read_pickle("BBB_Test_Decile6.pkl")
BBB_Test_Decile7 = pd.read_pickle("BBB_Test_Decile7.pkl")
BBB_Test_Decile8 = pd.read_pickle("BBB_Test_Decile8.pkl")
BBB_Test_Decile9 = pd.read_pickle("BBB_Test_Decile9.pkl")
BBB_Test_Decile10 = pd.read_pickle("BBB_Test_Decile10.pkl")

BBB_Train_Targets_1D = pd.read_pickle("BBB_Train_Targets_1D.pkl")
BBB_Test_Targets_1D = pd.read_pickle("BBB_Test_Targets_1D.pkl")

In [3]:
DDD_Train_Decile0 = pd.read_pickle("DDD_Train_Decile0.pkl")
DDD_Train_Decile1 = pd.read_pickle("DDD_Train_Decile1.pkl")
DDD_Train_Decile2 = pd.read_pickle("DDD_Train_Decile2.pkl")
DDD_Train_Decile3 = pd.read_pickle("DDD_Train_Decile3.pkl")
DDD_Train_Decile4 = pd.read_pickle("DDD_Train_Decile4.pkl")
DDD_Train_Decile5 = pd.read_pickle("DDD_Train_Decile5.pkl")
DDD_Train_Decile6 = pd.read_pickle("DDD_Train_Decile6.pkl")
DDD_Train_Decile7 = pd.read_pickle("DDD_Train_Decile7.pkl")
DDD_Train_Decile8 = pd.read_pickle("DDD_Train_Decile8.pkl")
DDD_Train_Decile9 = pd.read_pickle("DDD_Train_Decile9.pkl")
DDD_Train_Decile10 = pd.read_pickle("DDD_Train_Decile10.pkl")

DDD_Test_Decile0 = pd.read_pickle("DDD_Test_Decile0.pkl")
DDD_Test_Decile1 = pd.read_pickle("DDD_Test_Decile1.pkl")
DDD_Test_Decile2 = pd.read_pickle("DDD_Test_Decile2.pkl")
DDD_Test_Decile3 = pd.read_pickle("DDD_Test_Decile3.pkl")
DDD_Test_Decile4 = pd.read_pickle("DDD_Test_Decile4.pkl")
DDD_Test_Decile5 = pd.read_pickle("DDD_Test_Decile5.pkl")
DDD_Test_Decile6 = pd.read_pickle("DDD_Test_Decile6.pkl")
DDD_Test_Decile7 = pd.read_pickle("DDD_Test_Decile7.pkl")
DDD_Test_Decile8 = pd.read_pickle("DDD_Test_Decile8.pkl")
DDD_Test_Decile9 = pd.read_pickle("DDD_Test_Decile9.pkl")
DDD_Test_Decile10 = pd.read_pickle("DDD_Test_Decile10.pkl")

DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
DDD_Test_Targets_1D = pd.read_pickle("DDD_Test_Targets_1D.pkl")

In [4]:
FFF_Train_Decile0 = pd.read_pickle("FFF_Train_Decile0.pkl")
FFF_Train_Decile1 = pd.read_pickle("FFF_Train_Decile1.pkl")
FFF_Train_Decile2 = pd.read_pickle("FFF_Train_Decile2.pkl")
FFF_Train_Decile3 = pd.read_pickle("FFF_Train_Decile3.pkl")
FFF_Train_Decile4 = pd.read_pickle("FFF_Train_Decile4.pkl")
FFF_Train_Decile5 = pd.read_pickle("FFF_Train_Decile5.pkl")
FFF_Train_Decile6 = pd.read_pickle("FFF_Train_Decile6.pkl")
FFF_Train_Decile7 = pd.read_pickle("FFF_Train_Decile7.pkl")
FFF_Train_Decile8 = pd.read_pickle("FFF_Train_Decile8.pkl")
FFF_Train_Decile9 = pd.read_pickle("FFF_Train_Decile9.pkl")
FFF_Train_Decile10 = pd.read_pickle("FFF_Train_Decile10.pkl")

FFF_Test_Decile0 = pd.read_pickle("FFF_Test_Decile0.pkl")
FFF_Test_Decile1 = pd.read_pickle("FFF_Test_Decile1.pkl")
FFF_Test_Decile2 = pd.read_pickle("FFF_Test_Decile2.pkl")
FFF_Test_Decile3 = pd.read_pickle("FFF_Test_Decile3.pkl")
FFF_Test_Decile4 = pd.read_pickle("FFF_Test_Decile4.pkl")
FFF_Test_Decile5 = pd.read_pickle("FFF_Test_Decile5.pkl")
FFF_Test_Decile6 = pd.read_pickle("FFF_Test_Decile6.pkl")
FFF_Test_Decile7 = pd.read_pickle("FFF_Test_Decile7.pkl")
FFF_Test_Decile8 = pd.read_pickle("FFF_Test_Decile8.pkl")
FFF_Test_Decile9 = pd.read_pickle("FFF_Test_Decile9.pkl")
FFF_Test_Decile10 = pd.read_pickle("FFF_Test_Decile10.pkl")

FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
FFF_Test_Targets_1D = pd.read_pickle("FFF_Test_Targets_1D.pkl")